In [1]:
!pip install torch torchvision
!pip install -U finance-datareader


In [2]:
import pandas as pd
import pandas_datareader as pdr
import FinanceDataReader as fdr
import datetime
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import numpy as np
import argparse
import time
from copy import deepcopy # Add Deepcopy for args
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [35]:
x_frames = 7
start = datetime.datetime.now() - datetime.timedelta(days=(x_frames*3))
end = datetime.date.today()
df_du = pdr.DataReader('POILDUBUSDM', 'fred', start=datetime.datetime.now() - datetime.timedelta(days=(100)), end=end)#석유
df_gold = pdr.DataReader('GOLDAMGBD228NLBM', 'fred', start=start, end=end)#금값
df_kospi = fdr.DataReader('KS11', start, end)#코스피
df_dollar = fdr.DataReader('USD/KRW', start, end)#원달러환율

df_kospi['gold'] = df_gold
df_kospi['dollar'] = df_dollar['Close']
#oil 채우기
du = [df_du.values[-1] for i in range(x_frames)]

#gold 채우기
df_kospi = df_kospi.fillna(value=-1)
for i,val in enumerate(df_kospi['gold']):
    if val == -1:
        df_kospi['gold'][i] = df_kospi['gold'][i-1]
df_kospi = df_kospi.tail(x_frames)
df_kospi['oil'] = pd.DataFrame(data=du, index=df_kospi.index, columns=['oil'])


In [36]:
df_kospi

,Close,Open,High,Low,Volume,Change,gold,dollar,oil
Date,,,,,,,,,
2020-09-25,2278.79,2295.19,2296.36,2267.63,697960000.0,0.0027,1870.05,1174.32,43.705238
2020-09-28,2308.08,2301.16,2314.81,2291.32,605980000.0,0.0129,1850.95,1166.85,43.705238
2020-09-29,2327.89,2331.51,2339.83,2320.04,646220000.0,0.0086,1882.40,1168.73,43.705238
2020-10-05,2358.00,2330.55,2364.73,2327.83,763620000.0,0.0129,1899.65,1157.12,43.705238
2020-10-06,2365.90,2369.17,2378.28,2355.39,861750000.0,0.0034,1912.50,1164.37,43.705238
2020-10-07,2386.94,2350.82,2387.45,2347.82,737990000.0,0.0089,1888.00,1157.26,43.705238
2020-10-08,2391.96,2408.49,2409.01,2383.82,961730000.0,0.0021,1891.35,1151.11,43.705238


In [67]:
class predict_Dataset(Dataset):
    
    def __init__(self, symbol, x_frames,df_kospi):
        
        self.symbol = symbol
        self.x_frames = x_frames
        
        self.start = datetime.datetime.now() - datetime.timedelta(days=(x_frames*3))
        self.end = datetime.date.today()

        self.scaler = MinMaxScaler()
        self.data = fdr.DataReader(self.symbol, self.start, self.end)#주식 6가지 특징, 고,시,저,종,거,변
        
        if len(self.data.index) == 0:
            print(self.symbol,' is zero')
            return None
        self.data['dollar'] = df_dollar[['Close']]
        self.data['kospi'] = df_kospi[['Close']]
        self.data['oil'] = df_kospi[['oil']]
        self.data['gold'] = df_kospi[['gold']]
        self.data = pd.DataFrame(self.scaler.fit_transform(self.data))
        self.data = self.data.tail(x_frames)
        
    def __len__(self):
        return len(self.data) - x_frames + 1
    
    def __getitem__(self, idx):
        data = self.data.values
        X = data[:]
        
        return X

In [38]:
class GRU(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, batch_size, dropout, use_bn):
        super(GRU, self).__init__()
        self.input_dim = input_dim 
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.num_layers = num_layers

        self.batch_size = batch_size
        self.dropout = dropout
        self.use_bn = use_bn 
        self.gru = nn.GRU(self.input_dim, self.hidden_dim*2, self.num_layers)
        self.gru2 = nn.GRU(self.hidden_dim*2, self.hidden_dim, self.num_layers)
        self.regressor = self.make_regressor()

    def make_regressor(self):
        layers = []
        if self.use_bn:
            layers.append(nn.BatchNorm1d(self.hidden_dim))
        layers.append(nn.Dropout(self.dropout))
        
        layers.append(nn.Linear(self.hidden_dim, self.hidden_dim//2))
        layers.append(nn.ReLU())
        layers.append(nn.Linear(self.hidden_dim//2, self.output_dim))
        regressor = nn.Sequential(*layers)
        return regressor
    
    def forward(self, x):
        x, _ = self.gru(x)
        x, _ = self.gru2(x)
        y_pred = self.regressor(x[-1].view(self.batch_size, -1))
        return y_pred

In [39]:
#주식정보들
Symbols = []
with open('/content/gdrive/My Drive/Symbols.txt', 'r') as f:
    data = f.readline()
    Symbols = data.split(',')
print(len(Symbols))

795


In [40]:
batch_size = 1
input_dim = 10
hidden_dim = 64
num_layers = 2
input_frame = 7
output_frame = 7
dropout = 1.0
use_bn = True

model = GRU(input_dim,hidden_dim,output_frame,num_layers,batch_size,dropout,use_bn)
modelPath = '/content/gdrive/My Drive/GRUmodel/GRU_Best_model.pt'
model.load_state_dict(torch.load(modelPath))
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model.to(device)

GRU(
  (gru): GRU(10, 128, num_layers=2)
  (gru2): GRU(128, 64, num_layers=2)
  (regressor): Sequential(
    (0): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Dropout(p=1.0, inplace=False)
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=7, bias=True)
  )
)

In [41]:
df_gold

,GOLDAMGBD228NLBM
DATE,
2020-09-21,1930.90
2020-09-22,1903.10
2020-09-23,1888.10
2020-09-24,1850.75
2020-09-25,1870.05
2020-09-28,1850.95
2020-09-29,1882.40
2020-09-30,1883.40
2020-10-01,1895.55


In [42]:
df_kospi

,Close,Open,High,Low,Volume,Change,gold,dollar,oil
Date,,,,,,,,,
2020-09-25,2278.79,2295.19,2296.36,2267.63,697960000.0,0.0027,1870.05,1174.32,43.705238
2020-09-28,2308.08,2301.16,2314.81,2291.32,605980000.0,0.0129,1850.95,1166.85,43.705238
2020-09-29,2327.89,2331.51,2339.83,2320.04,646220000.0,0.0086,1882.40,1168.73,43.705238
2020-10-05,2358.00,2330.55,2364.73,2327.83,763620000.0,0.0129,1899.65,1157.12,43.705238
2020-10-06,2365.90,2369.17,2378.28,2355.39,861750000.0,0.0034,1912.50,1164.37,43.705238
2020-10-07,2386.94,2350.82,2387.45,2347.82,737990000.0,0.0089,1888.00,1157.26,43.705238
2020-10-08,2391.96,2408.49,2409.01,2383.82,961730000.0,0.0021,1891.35,1151.11,43.705238


In [70]:
predicts = {}
i = 0
for sb in Symbols:
    i += 1
    if i % 100 == 0:
        print(i)
    pset = predict_Dataset(sb,input_frame,df_kospi)
    model.eval()
    loader = DataLoader(pset,batch_size,shuffle=False)
    for X in loader:
        X = X.transpose(0, 1).float().to(device)
        y_pred = model(X)
        y_pred = y_pred.view(-1)
        predicts[sb] = y_pred[-1]


tensor([[[0.0000, 0.0000, 0.0000, 0.0556, 0.1216, 0.6243, 1.0000, 0.0000,
          0.0000, 0.3103],
         [0.0769, 0.3200, 0.2632, 0.5556, 0.0238, 1.0000, 0.6782, 0.2588,
          0.0000, 0.0000],
         [0.3846, 0.4400, 0.5789, 0.5000, 0.0000, 0.5321, 0.7592, 0.4339,
          0.0000, 0.5110],
         [0.3077, 0.3200, 0.5263, 0.5556, 0.0470, 0.6223, 0.2589, 0.6999,
          0.0000, 0.7912],
         [0.4615, 0.4800, 0.5263, 0.5556, 0.2159, 0.5771, 0.5713, 0.7697,
          0.0000, 1.0000],
         [0.1923, 0.2400, 0.4737, 0.5000, 0.1675, 0.5321, 0.2650, 0.9556,
          0.0000, 0.6019],
         [0.5000, 0.4800, 0.6316, 0.6111, 0.4159, 0.6675, 0.0000, 1.0000,
          0.0000, 0.6564]]], dtype=torch.float64)
tensor([0.5711, 0.5724, 0.5716, 0.5693, 0.5694, 0.5668, 0.5674],
       device='cuda:0', grad_fn=<ViewBackward>)


In [71]:
print(predicts)

{'353200': tensor(0.5674, device='cuda:0', grad_fn=<SelectBackward>)}


In [ ]:
!pip install pymysql
import pymysql

In [ ]:
conn = pymysql.connect(host='jukerdb.cwhsnjoqybdo.ap-northeast-2.rds.amazonaws.com', user='admin', password='',
                       db='WEB', charset='utf8')
 
# Connection 으로부터 Cursor 생성
curs = conn.cursor()
 

for sb in result:
    #table : stockpreidct
    #symbol, value, type, day
    # SQL문 실행
    sql = """INSERT INTO stockpredict VALUES(%s,%s,%s,NOW())"""
    val = (sb,result[sb],'L')
    curs.execute(sql,val)
    conn.commit()

sql = "select * from stockpredict"
curs.execute(sql)
rows = curs.fetchall()
print(rows)     # 전체 rows

# Connection 닫기
conn.close()